# Стохастический градиентный и координатный спуски

Для каждого задания указано количество баллов (если они оцениваются отдельно) + 1 балл за аккуратное и полное выполнение всего задания

## Загрузка и подготовка данных

**Загрузите уже знакомый вам файл *Advertising.csv* как объект DataFrame.** 

In [418]:
import pandas as pd
import numpy as np


df = pd.read_csv('data/Advertising.csv', index_col=0)
display(df.head())
display(f'Размерность DataFrame: {df.shape}')

,TV,radio,newspaper,sales
1,230.1,37.8,69.2,22.1
2,44.5,39.3,45.1,10.4
3,17.2,45.9,69.3,9.3
4,151.5,41.3,58.5,18.5
5,180.8,10.8,58.4,12.9


'Размерность DataFrame: (200, 4)'

**Проверьте, есть ли в данных пропуски и, если они есть - удалите их**

In [419]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 200 entries, 1 to 200
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   TV         200 non-null    float64
 1   radio      200 non-null    float64
 2   newspaper  200 non-null    float64
 3   sales      200 non-null    float64
dtypes: float64(4)
memory usage: 7.8 KB


In [420]:
df.isnull().sum()

TV           0
radio        0
newspaper    0
sales        0
dtype: int64

**Преобразуйте ваши признаки в массивы NumPy и разделите их на переменные X (предикторы) и y (целевая переменная)** 

In [421]:
X = df.drop('sales', axis=1).values
y = df['sales'].values

## Координатный спуск (3 балла)

**Добавим единичный столбец для того, чтобы у нас был свободный коэффициент в уравнении регрессии:**

In [422]:
X = np.hstack([np.ones(X.shape[0]).reshape(-1, 1), X])
y = y.reshape(-1, 1)
print(X.shape, y.shape)

(200, 4) (200, 1)


**Нормализуем данные: обычно это необходимо для корректной работы алгоритма**

In [423]:
X = X / np.sqrt(np.sum(np.square(X), axis=0))

**Реализуйте алгоритм координатного спуска:** (3 балла)

Ниже приведен алгоритм координатного спуска для случая нормализованных данных:

**Задано:**

* $X=(x_{ij})$ - матрица наблюдений, размерностью $dim(X)=(n, m)$
* $N=1000$ - количество итераций

**Примечание:** *1000 итераций здесь указаны для этого задания, на самом деле их может быть намного больше, нет детерменированного значения.*

**Алгоритм (математическая запись):**
* Создать нулевой вектор параметров $w_0=(0, 0,..., 0)^T$
* Для всех $t=1, 2, ..., N$ итераций:
    * Для всех $k = 1, 2,..., m$:
        * Фиксируем значение всех признаков, кроме $k$-ого и вычисляем прогноз модели линейной регрессии.Для этого исключаем признак $k$-ый из данных и $w_j$ из параметров при построении прогноза.
        Математически это можно записать следующим образом:

        $$h_i = \sum_{j=1}^{k-1} x_{ij}w_{j} + \sum_{j=k+1}^{m} x_{ij}w_j $$

        **Примечание:**
        
        *Обратите, что в данной записи текущий признак под номером $k$ не участвует в сумме.Сравните эту запись с классической записью прогноза линейной регрессии в случае нормированных данных (когда участвуют все признаки):*

        $$h_i = \sum_{j=1}^{m} x_{ij}w_{j}$$ 
        
        * Вычисляем новое значение параметра $k$-ого коэффициента: 
        $$w_k = \sum_{i=1}^{n} x_{ik} (y_i - h_i) = x_k^T(y-h) $$

    * Вычисляем значение функции потерь и сохраняем в историю изменения функции потерь (В оценке функции потерь участвуют все признаки):
        $$\hat{y_i} = \sum_{j=1}^{m}x_{ij}w_j$$
        $$Loss_t = \frac{1}{n} \sum_{i=1}^{n}(y_i-\hat{y_i})^2$$
        
        или в векторном виде:
        
        $$\hat{y} = Xw$$
        $$Loss_t = \frac{1}{n}(y-\hat{y})^T(y-\hat{y})$$
    



**Алгоритм (псевдокод):**
```python

num_iters = #количество итераций
m = # количество строк в матрице X
n = # количество столбцов в матрице X
w = #вектор размера nx1, состояющий из нулей

for i in range(num_iters):
    for k in range(n):
        # Вычисляем прогноз без k-ого фактора
        h = (X[:,0:k] @ w[0:k]) + (X[:,k+1:] @ w[k+1:])
        # Обновляем новое значение k-ого коэффициента
        w[k] =  (X[:,k].T @ (y - h))
        # Вычисляем функцию потерь
        cost = sum((X @ w) - y) ** 2)/(len(y))

```

Вам необходимо реализовать координатный спуск, и вывести веса в модели линейной регрессии.

In [424]:
num_iters = 1000 # количество итераций
n = X.shape[0] # количество строк в матрице X
m = X.shape[1] # количество столбцов в матрице X
w = np.zeros((m, 1), dtype=float)  # вектор размера mx1, состояющий из нулей - начальный вектор весов
cost_lst = [] # список для сохранения значений функции потерь

for i in range(num_iters):
    for k in range(m):
        # вычисляем прогноз без k-ого фактора
        h = (X[:,0:k] @ w[0:k]) + (X[:,k+1:] @ w[k+1:])
        # вычисляем новое значение k-ого коэффициента
        w[k] =  (X[:,k].T @ (y - h))
        # вычисляем функцию потерь и сохраняем ее значение
        cost = (y - X @ w).T @ (y - X @ w) / n
        cost_lst.append(cost)
        
print(f'Веса в модели линейной регрессии по методу координетного спуска: {np.round(w.T[0], 8)}')

Веса в модели линейной регрессии по методу координетного спуска: [ 41.56217205 110.13144155  73.52860638  -0.55006384]


Сравните результаты с реализацией линейной регрессии из библиотеки sklearn:

In [425]:
from sklearn.linear_model import LinearRegression
 
model = LinearRegression(fit_intercept=False)
model.fit(X, y)
 
print(f'Веса в модели линейной регрессии из библиотеки sklearn: {np.round(model.coef_[0], 8)}')

Веса в модели линейной регрессии из библиотеки sklearn: [ 41.56217205 110.13144155  73.52860638  -0.55006384]


Если вы все сделали верно, они должны практически совпасть!

In [426]:
# сравним веса модели вычисленные методом 
# координатного спуска и с помощью библиотеки sklearn
np.round(w.T[0], 8) - np.round(model.coef_[0], 8)

array([0., 0., 0., 0.])

In [427]:
# выясним на какой итерации значение функции потерь 
# практически перестает изменяться и вычислим разницу 
# между значениями функции потерь на этой итерации и 1000 итерации

val = 0.00000001 # порог изменения функции потерь на двух соседних итерациях

for i in range(1, num_iters):
    if abs(cost_lst[i] - cost_lst[i-1]) < val:
        print(f'Необходимо {i} итераций для достижения изменения функции потерь менее {val}')
        print(f'Значение функции потерь на {i} итерации: {np.round(cost_lst[i][0][0], 8)}')
        print(f'Разница значений функции потерь на {i} и 1000 итерациях: {np.round(cost_lst[i][0][0] - cost_lst[-1][0][0], 8)}')
        break


Необходимо 85 итераций для достижения изменения функции потерь менее 1e-08
Значение функции потерь на 85 итерации: 2.78462742
Разница значений функции потерь на 85 и 1000 итерациях: 0.00050111


## Стохастический градиентный спуск (6 баллов)

**Отмасштабируйте столбцы исходной матрицы *X* (которую мы не нормализовали еще!). Для того, чтобы это сделать, надо вычесть из каждого значения среднее и разделить на стандартное отклонение** (0.5 баллов)

In [428]:
X = df.drop('sales', axis=1).values
 
X = (X - np.mean(X, axis=0)) / np.std(X, axis=0)

**Добавим единичный столбец**

In [429]:
X = np.hstack([np.ones(X.shape[0]).reshape(-1, 1), X])

**Создайте функцию mse_error для вычисления среднеквадратичной ошибки, принимающую два аргумента: реальные значения и предсказывающие, и возвращающую значение mse** (0.5 балла)

In [430]:

def mse_error(y_true, y_pred):
    return np.mean((y_true - y_pred)**2)

**Сделайте наивный прогноз: предскажите продажи средним значением. После этого рассчитайте среднеквадратичную ошибку для этого прогноза** (0.5 балла)

In [431]:
# массив среднего значения
y_mean = np.full((y.shape[0], 1), np.mean(y))

print(f'MSE для предсказания со средним значением продаж: {round(mse_error(y, y_mean), 8)}')

MSE для предсказания со средним значением продаж: 27.08574375


**Создайте функцию *lin_pred*, которая может по матрице предикторов *X* и вектору весов линейной модели *w* получить вектор прогнозов** (0.5 балла)

In [432]:
def lin_pred(X, w):
    return X @ w

**Создайте функцию *stoch_grad_step* для реализации шага стохастического градиентного спуска. (1.5 балла) 
Функция должна принимать на вход следующие аргументы:**
* матрицу *X*
* вектора *y* и *w*
* число *train_ind* - индекс объекта обучающей выборки (строки матрицы *X*), по которому считается изменение весов
* число *$\eta$* (eta) - шаг градиентного спуска

Результатом будет вектор обновленных весов

Шаг для стохастического градиентного спуска выглядит следующим образом:

$$\Large w_j \leftarrow w_j - \frac{2\eta}{\ell} \sum_{i=1}^\ell{{x_{ij}((w_0 + w_1x_{i1} + w_2x_{i2} +  w_3x_{i3}) - y_i)}}$$

Для того, чтобы написать функцию, нужно сделать следующее:
    
*  посчитать направление изменения: умножить объект обучающей выборки на 2 и на разницу между предсказанным значением и реальным, а потом поделить на количество элементов в выборке.
* вернуть разницу между вектором весов и направлением изменения, умноженным на шаг градиентного спуска

In [433]:
def stoch_grad_step(X, y, w, train_ind, eta):
    step_gd = 2 * (lin_pred(X[train_ind, :], w) - y[train_ind]) * X[train_ind, :].reshape(-1, 1)
    return w - eta * step_gd

**Создайте функцию *stochastic_gradient_descent*, для реализации стохастического градиентного спуска (2.5 балла)**

**Функция принимает на вход следующие аргументы:**
- Матрицу признаков X
- Целевую переменнную
- Изначальную точку (веса модели)
- Параметр, определяющий темп обучения
- Максимальное число итераций
- Евклидово расстояние между векторами весов на соседних итерациях градиентного спуска,при котором алгоритм прекращает работу 

**На каждой итерации в вектор (список) должно записываться текущее значение среднеквадратичной ошибки. Функция должна возвращать вектор весов $w$, а также вектор (список) ошибок.**

Алгоритм сследующий:
    
* Инициализируйте расстояние между векторами весов на соседних итерациях большим числом (можно бесконечностью)
* Создайте пустой список для фиксации ошибок
* Создайте счетчик итераций
* Реализуйте оновной цикл обучения пока расстояние между векторами весов больше того, при котором надо прекратить работу (когда расстояния станут слишком маленькими - значит, мы застряли в одном месте) и количество итераций меньше максимально разрешенного: сгенерируйте случайный индекс, запишите текущую ошибку в вектор ошибок, запишите в переменную текущий шаг стохастического спуска с использованием функции, написанной ранее. Далее рассчитайте текущее расстояние между векторами весов и прибавьте к счетчику итераций 1.
* Верните вектор весов и вектор ошибок

In [434]:
def stochastic_gradient_descent(X, y, w, eta, num_iter, stop_dist):
    euc_dist = stop_dist + 1 # начальное евклидово расстояние между векторами весов на соседних итерациях (гарантированно больше заданного условия остановки алгоритма)
    mse_lst = [mse_error(y, lin_pred(X, w))] # список для фиксации значений ошибок MSE, первое значение для начального вектора весов
    for i in range(num_iter): # цикл с заданным числом итераций num_iter
        if euc_dist < stop_dist: # условие прекращения цикла при расстоянии между векторами весов на соседних итерациях меньше заданного stop_dist
            print(f'Итерация, на которой произошел останов алгоритма: {i}')
            break
        train_ind = np.random.randint(0, y.shape[0]) # генерация случайного индекса для шага стохастического градиентного спуска
        w_new = stoch_grad_step(X, y, w, train_ind, eta) # вычисляем новый вектор весов (один шаг стохастического градиентного спуска)
        mse_lst.append(mse_error(y, lin_pred(X, w_new))) # вычисление ошибки MSE для нового вектора весов и добавление в список
        euc_dist = np.sqrt(np.sum((w_new - w)**2)) # вычисление евклидова расстояния между новым и старым векторами весов
        w = w_new # обновляем вектор весов
    return w, mse_lst

 **Запустите $10^5$ итераций стохастического градиентного спуска. Укажите вектор начальных весов, состоящий из нулей. Можете поэкспериментировать с параметром, отвечающим за темп обучения.**

In [435]:
w = np.zeros((m, 1), dtype=float)  # вектор размера mx1, состояющий из нулей - начальный вектор весов
num_iter = 10**5 # количество итераций
eta = 0.001 # темп обучения
stop_dist = 0.00001 # евклидово расстояние между векторами весов на соседних итерациях стохастического градиентного спуска, при котором алгоритм прекращает работу 

w, mse_lst = stochastic_gradient_descent(X, y, w, eta, num_iter, stop_dist)

Итерация, на которой произошел останов алгоритма: 3942


**Выведите вектор весов, к которому сошелся метод.**

In [436]:
print(w)

[[14.00816706]
 [ 3.88366781]
 [ 2.82328089]
 [ 0.01610203]]


**Выведите среднеквадратичную ошибку на последней итерации.**

In [437]:
print(round(mse_lst[-1], 8))

2.78864251


**Постройте график зависимости ошибки от номера итерации**

In [438]:
import plotly.graph_objects as go


fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(len(mse_lst)), 
                         y=mse_lst, 
                         name='MSE', 
                         mode='markers'))
fig.update_traces(hoverinfo="all", hovertemplate="iteration: %{x}<br>valume: %{y}")
fig.update_layout(title = 'Dependence of the MSE value on the iteration number', 
                  title_x = 0.5, title_font_size=20)
fig.update_xaxes(title_font=dict(size=16, family='Rockwell',), 
                 title_text='Iteration number',
                 tickfont=dict(family='Rockwell', size=14))
fig.update_yaxes(title_font=dict(size=16, family='Rockwell'), 
                 title_text='MSE',
                 tickfont=dict(family='Rockwell', size=14))
fig.show()

Сравним результаты с реализацией линейной регрессии из библиотеки sklearn:


In [439]:
# используем те же самые X и y (матрицу признаков и 
# целевую переменную)
model = LinearRegression(fit_intercept=False)
model.fit(X, y)
print(f'Веса в модели линейной регрессии из библиотеки sklearn: {np.round(model.coef_[0], 8)}')
# и вычислим MSE - среднеквадратичную ошибку
y_pred = lin_pred(X, model.coef_.reshape(-1, 1))
print(f'MSE для предсказания со средним значением продаж: {round(mse_error(y, y_pred), 8)}')

Веса в модели линейной регрессии из библиотеки sklearn: [14.0225      3.91925365  2.79206274 -0.02253861]
MSE для предсказания со средним значением продаж: 2.78412631


#### **Выводы:**

1. Метод координатного спуска дает результат, совпадающий по точности с результатом при использовании модели линейной регрессии библиотеки Scikit-learn.

Замечание: количество итераций для метода координатного спуска = 1000, метрика ошибки MSE (среднеквадратичная ошибка). Совпадение с результатом Scikit-learn точное (до 8-10 знаков после запятой).

2. Метод стохастического градиентного спуска дает результат, совпадающий по точности с результатом при использовании модели линейной регрессии библиотеки Scikit-learn. 

Замечание: выбран темп обучение 0.001 и условие прекращения - расстояние между векторами весов соседних итераций менее 0.00001, метрика ошибки MSE (среднеквадратичная ошибка). Для таких условий достаточно 1000 - 5000 итераций для достижения результата. Совпадение с результатом Scikit-learn достаточно точное (2 знак после запятой).

Замечание: повторяемость результата не гарантирована.


**Замечание:** при использовании метода стохастического градиентного спуска лучше выбирать вектор весов, соответствующий минимальному значению MSE. Т.е. сохранять пары (вектор весов, значение MSE) для каждой итерации и затем выбирать по минимальному значению MSE. Так же этот метод может быть неустойчив в смысле зависимости от темпа обучения.
